In [1]:
# To Do
# Succesfully open and read one of the data files
# Inspect the columns and their data types
#Append the remaining datasets together
# Remove any columns and records that will not suit the investigation
# Write code that creates the table for the data
# Upload the data to the table

In [2]:
# Importing Dependencies
import pandas as pd
import os

### Cleaning the WHO Mortality Dataset
* Contains records from 1988 to 2020 on causes of death
* Some records in the "Sex" column are unspecified, coded as a 9. For our purposes we will remove these records
* 

In [3]:

# Cycle through the different parts of the Morticd and combine them into a single dataframe
dirLoc = 'Resources' #filepath of the directory
df = pd.DataFrame()

for file in os.scandir(dirLoc):
    if "Morti" in file.path: # Only append files from the Mortality Datasets
        df = df.append(pd.read_csv(file.path,))

        
        

C:\Users\daypa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\daypa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [27]:
# Removing records with value 9
df["Sex"].value_counts() # There are 20,701 records with the 9 value

# Creating new dataframe wihtout 9 values in the Sex field
df_Removed9 = df[df["Sex"] != 9]

df_Removed9["Sex"].value_counts() # 9 values removed from the original dataset

1    2358126
2    2127394
Name: Sex, dtype: int64

In [29]:
#Removing uneccesary columns
#df_excised_cols = df_Removed9[df_Removed9[["Country","Year","Cause","Sex","Deaths1"]]]# Takes too long to process consider a big data solution?

KeyboardInterrupt: 

In [5]:
minYear = min(df['Year'])
maxYear = max(df['Year'])
print(minYear , maxYear)
# Observing the Scope of the Mortality Dataset  (1988 - 2020)
# Scope of the Meat Consumption ddDataset is 1990 - 2029
# We can limit the size of the datsets within the years 2000 and 2020
# Need to consider ways of improving write speed of Dataframe to PSQL database
# One option is to split the 2000 - 2020 datset by decade and upload that.
# Inspect both datasets and clean them so only the essential columns remain, consider using an ERD to do this

1988 2020
